<a href="https://colab.research.google.com/github/kconstable/market_predictions/blob/main/market_data_av.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
# import sys
# !{sys.executable} -m pip install plotly

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import pickle
import re
from datetime import date, timedelta,datetime
from tabulate import tabulate
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from google.colab import files

# alphavalue key
key = '2Y4KISB4XMRRRPW7'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Cypto Currencies

In [24]:
def get_crypto_intraday(symbol,key,interval='60min',outputsize='compact',market='USD'):
  """
  CRYPTO_INTRADAY
    symbol: DOGE, BTC, ETH
    key: apikey
    interval: 1min, 5min, 15min, 30min, 60min
    outputsize: compact(last 100), full (all)
    market: USD
  """

  # build query string, get data as json and convert to a dict
  url = f"https://www.alphavantage.co/query?function=CRYPTO_INTRADAY&symbol={symbol}&market={market}&interval={interval}&outputsize={outputsize}&apikey={key}"
  r = requests.get(url)
  d = r.json()

  # extract data to df
  df=pd.DataFrame(d[f'Time Series Crypto ({interval})']).T
  df.columns = ['open','high','low','close','volume']
  df['symbol'] = d['Meta Data']['2. Digital Currency Code']
  df['name'] = d['Meta Data']['3. Digital Currency Name']

  return df


In [39]:
def get_crypto_data(symbol,key):
  """
  'ETH', 'BTC', 'DOGE'
  """
  # build query string, get data as json and convert to a dict
  url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market=CAD&apikey={key}'
  r = requests.get(url)
  d = r.json()

  # extract data to df
  df=pd.DataFrame(d['Time Series (Digital Currency Daily)']).T

  # remove columns not required
  # returns the price in two currencies, just keep USD
  cols = [c for c in df.columns if '(CAD)' not in c]
  df=df.loc[:, cols]
  df.columns = ['open','high','low','close','volume','marketcap']
  df.drop(['marketcap'],axis=1,inplace=True)

  # change data types
  df.index = pd.to_datetime(df.index)

  # convert datatype to float
  for col in ['open','high','low','close','volume']:
    df[col] = df[col].astype('float')

  # add the cyrpto name
  df['symbol'] = d['Meta Data']['3. Digital Currency Name']

  return df
   

### Technical Indicators


#### Bollinger Bands

In [97]:
def calc_bollinger(df,feature,window=20,st=2):
  """
  """

  # rolling mean and stdev
  rolling_m  = df[feature].rolling(window).mean()
  rolling_st = df[feature].rolling(window).std()

  # add the upper/lower and middle bollinger bands
  df['b-upper']  = rolling_m + (rolling_st * st)
  df['b-middle'] = rolling_m 
  df['b-lower']  = rolling_m - (rolling_st * st)


#### Relative Strength Index (RSI)-leading indicator

In [127]:
def calc_rsi(df,feature='close',window=14):
  """
  """
  # RSI
  # calc the diff in daily prices, exclude nan
  diff =df[feature].diff()
  diff.dropna(how='any',inplace=True)

  # separate positive and negitive changes
  pos_m, neg_m = diff.copy(),diff.copy()
  pos_m[pos_m<0]=0
  neg_m[neg_m>0]=0

  # positive/negative rolling means
  prm = pos_m.rolling(window).mean()
  nrm = neg_m.abs().rolling(window).mean()

  # calc the rsi and add to the df
  ratio = prm /nrm
  rsi = 100.0 - (100.0 / (1.0 + ratio))
  df['rsi']=rsi

#### Moving Average Convergence/Divergence (MACD)

In [84]:
def calc_macd(df,feature='close'):
  """
  """
  ema12 = df[feature].ewm(span=12,adjust=False).mean()
  ema26 = df[feature].ewm(span=26,adjust=False).mean()
  df['macd']=ema12-ema26
  df['macd_signal'] = df['macd'].ewm(span=9,adjust=False).mean()


### Combine Cyrpto Data

In [119]:
def get_consolidated_crypto_data(symbol,key,config,boll_window=20,boll_std=2,rsi_window=14,throttle=30,dropna=True):
  """
  Pulls data from alpha advantage and consolidates
  API Limitations: 5 API requests per minute and 500 requests per day
  Inputs:
    symbol: stock ticker
    key   : api key
    config: dictionary which lists the economic, technical and commodities to pull
    outputsize: compact(latest 100) or full (up to 20 years of daily data)
    throttle: number of seconds to wait between api requests
    dropna: True/False, drops any records with nan
  Output:
    A dataframe with consolidated price data for the symbol + economic/technical
    indicators and commodity prices
  """

  # Result header and accumulator
  header = ['Type','Data','Total','Missing',' % ','Action']
  summary =[]

  # Get crypto prices
  try:
    dff  = get_crypto_data(symbol,key)
    print(f'Complete:===>Crypto:{symbol}')
  except:
    print(f'Error:===>Crypto:{symbol}')


  # Get Commodity prices
  # ****************************************************************************
  for commodity in config['Commodities']:
    try:
      # get prices
      results = get_ticker_data(commodity,key,'full',throttle)
      df = results['data']
      summary.append(results['summary'])
      print(f'Complete:===>Commodity:{commodity}')


      # rename close to commodity name, remove unneeded columns and join with 
      # the stock prices by date
      df.rename(columns={'close':commodity},inplace=True)
      df.drop(['open','high','low','volume','symbol'],axis=1,inplace=True)
      dff = dff.join(df,how='left')
    except:
      print(f"Error===>Commodity:{commodity}")


  # Economic Indicators
  # ****************************************************************************
  # loop through the config to pull the requested data
  for indicator,values in config['Economic'].items():
    if indicator == 'TREASURY_YIELD':
      for tr in values:
        try:
          results = get_economic_indicators(indicator,key,interval=tr['interval'],maturity=tr['maturity'],throttle=throttle)
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}:{tr['maturity']}")

          df = results['data']
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": tr['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}:{tr['maturity']}")
   
    else: 
      # daily
      if values['interval']=='daily':
        try:
      
          results = get_economic_indicators(indicator,key,interval=values['interval'],throttle=throttle)
          df = results['data']
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}")

          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")
  
      else: 
        try:
          # monthly or weekly
         
          results = get_economic_indicators(indicator,key,throttle=throttle)
          summary.append(results['summary'])
          df = results['data']
          print(f"Complete:===>{indicator}")

          # reindex to daily, fill missing values forward
          days = pd.date_range(start = min(df.index),end =max(df.index),freq='D')
          df =df.reindex(days,method = 'ffill')
      
          # join with the other data
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")

  # # Technical Indicators
  # ****************************************************************************
  calc_rsi(dff,'close',rsi_window)
  calc_bollinger(dff,'close',boll_window,boll_std)
  calc_macd(dff,'close')
      

  # Fill in any missing data after joining all datasets
  dff.fillna(method='bfill',inplace=True,axis = 0)

  # drop rows with missing commodity prices
  if dropna:
    dff.dropna(how='any',inplace=True)

  # print the results table
  print("\n\n")
  print(tabulate(summary,header))

  return dff

In [138]:
dff.shape

(998, 26)

### Pull Crypto Data

In [120]:
# Get Consolidated Data
# https://etfdb.com/
# cypto:  BTC, ETH,DOGE
# S&P500: SPY = jan 1993
# gold:   GLD = nov 2004
# oil:    USO =apr 2006, OIL = apr 2011
# energy: PBD =  jun 2007 (clean), PBW=mar 2005 (cleaner oil & gas), XLE= dec 1998 (oil & gas)
# gas:    UNL = jan 2010, BOIL=oct 2011
# vol:    VXX = jan 2009



config ={'Economic':
         {'TREASURY_YIELD':[{'interval':'daily','maturity':'5year','name':'yield5y'},
                            {'interval':'daily','maturity':'10year','name':'yield10y'},
                            {'interval':'daily','maturity':'30year','name':'yield30y'},
                            {'interval':'daily','maturity':'3month','name':'yield3m'}
                            ],
          'FEDERAL_FUNDS_RATE':{'interval':'daily','name':'ir'},
          'NONFARM_PAYROLL':{'interval':'monthly','name':'nfp'},
          'UNEMPLOYMENT':{'interval':'monthly','name':'unemployment'},
          'CONSUMER_SENTIMENT':{'interval':'monthly','name':'cs'},
          'INFLATION_EXPECTATION':{'interval':'monthly','name':'infl'},
          },
         'Commodities':['GLD','OIL','SPY','XLE','VXX','BOIL']
         }

# compact, full
cryto = 'ETH'
dff = get_consolidated_crypto_data(cryto,key,config,throttle=30,dropna=True)



# save to google drive
dff.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{crypto}_market_data.pickle')

# save to local drive
dff.to_pickle(f"{crypto}_market_data_{outputsize}.csv")
dff.to_csv(f'{crypto}_market_data_{outputsize}.pickle')
files.download(f"{crypto}_market_data_{outputsize}.csv")
files.download(f'{crypto}_market_data_{outputsize}.pickle')

Complete:===>Crypto:ETH
Complete:===>Commodity:GLD
Complete:===>Commodity:OIL
Complete:===>Commodity:SPY
Complete:===>Commodity:PBD
Complete:===>Commodity:VXX
Complete:===>TREASURY_YIELD:5year
Complete:===>TREASURY_YIELD:10year
Complete:===>TREASURY_YIELD:30year
Complete:===>TREASURY_YIELD:3month
Complete:===>FEDERAL_FUNDS_RATE
Complete:===>NONFARM_PAYROLL
Complete:===>UNEMPLOYMENT
Complete:===>CONSUMER_SENTIMENT
Complete:===>INFLATION_EXPECTATION



Type                Data                     Total    Missing   %      Action
------------------  ---------------------  -------  ---------  ------  -----------
Ticker              GLD                       4236          0  0.0%    none
Ticker              OIL                       2619          0  0.0%    none
Ticker              SPY                       5505          0  0.0%    none
Ticker              PBD                       3592          0  0.0%    none
Ticker              VXX                       3180          0  0.0%    none
Econ

# Stocks

In [25]:
def get_ticker_data(symbol,key,outputsize='compact',throttle=0):
  """
  Returns daily data for a stock (symbol)
    outputsize: compact(last 100) or full (20 years)
    key: apikey
    symbols: OILK (oil ETF),BAR(gold ETF),VXZ (volatility ETF)
  """
  url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize={outputsize}&apikey={key}'
  r = requests.get(url)
  time.sleep(throttle)
  d = r.json()

  # extract data to a df
  df = pd.DataFrame(d['Time Series (Daily)']).T
  df.columns = ['open','high','low','close','volume']
  df['symbol'] = d['Meta Data']['2. Symbol']

  # change data types
  df.index = pd.to_datetime(df.index)

  # convert datatype to float
  for col in ['open','high','low','close','volume']:
    df[col] = df[col].astype('float')

  # Calculate missing data
  missing = sum(df.close.isna())
  total = df.shape[0]
  missing_pct = round(missing/total*100,2)

  # Print the results
  summary = ['Ticker',symbol,str(total),str(missing),str(missing_pct)+'%','none']

  return {'summary':summary,'data':df}

## Economic Indicators

In [114]:
def get_economic_indicators(funct,key,interval=None,maturity=None,throttle=0):
  """
  Returns Economic Indicator Data with missing values interpolated between dates
  Monthly Data:
    NONFARM_PAYROLL, INFLATION_EXPECTATION,CONSUMER_SENTIMENT,UNEMPLOYMENT
  Daily, Weekly, Monthly Data:  
    FEDERAL_FUNDS_RATE = interval (daily,weekly,monthly)
    TREASURY_YIELD = interval (daily, weekly, monthly), 
                     maturity (3month, 5year, 10year, and 30year)
  """
  
  # query strings
  # Monthly Data:
  if funct in ['NONFARM_PAYROLL','INFLATION_EXPECTATION','CONSUMER_SENTIMENT','UNEMPLOYMENT']:
    url = f'https://www.alphavantage.co/query?function={funct}&apikey={key}'

  # Daily, Weekly or Monthly Data:
  # Interest Rates
  if funct == 'FEDERAL_FUNDS_RATE':
    url = f'https://www.alphavantage.co/query?function={funct}&interval={interval}&apikey={key}'

  # Treasury Yield  
  if funct == 'TREASURY_YIELD':
    url = f'https://www.alphavantage.co/query?function={funct}&interval={interval}&maturity={maturity}&apikey={key}'

  # pull data
  r = requests.get(url)
  time.sleep(throttle)
  d = r.json()

  # convert to df
  df = pd.DataFrame(d['data'])

  # move date to a datetime index
  df.date = pd.to_datetime(df.date)
  df.set_index('date',inplace=True)

  # add the ticker name and frequency
  df['name'] = d['name']
  df['interval']=d['interval'] 

  # clean data & interpolate missing values
  # missing data encoded with '.'
  # change datatype to float
  df.replace('.',np.nan,inplace=True)
  df.value = df.value.astype('float')

  # missing data stats
  missing =sum(df.value.isna())
  total =df.shape[0]
  missing_pct = round(missing/total*100,2)

  # interpolate using the time index
  if missing >0:
    df.value.interpolate(method='time',inplace=True)
    action = 'interpolate'
  else:
    action = 'none'

  # Print the results
  if maturity is not None:
    summary = ['Economic Indicator',funct+':'+maturity,str(total),str(missing),str(missing_pct)+'%',action]
  else:
    summary = ['Economic Indicator',funct,str(total),str(missing),str(missing_pct)+'%',action]


  return {'summary':summary,'data':df}


## Technical Indicators

In [5]:
def get_technical_indicators(symbol,funct,key,interval,time_period=None,throttle=0):
  """
  Returns Technical Indicators (only works for stocks, not cyrpto)
  MACD:   symbol,interval
  RSI:    symbol,interval,time_period
  BBANDS: symbol,interval,time_period

  Parameters:
          interval: (1min, 5min, 15min, 30min, 60min, daily, weekly, monthly)
          series_type: (open, close,high,low)-default to close
          timer_periods: Integer
  """
  # build the query string
  if funct =='MACD':
    url = f'https://www.alphavantage.co/query?function={funct}&symbol={symbol}&interval={interval}&series_type=close&apikey={key}'
  if funct in ['RSI','BBANDS']:
    url = f'https://www.alphavantage.co/query?function={funct}&symbol={symbol}&interval={interval}&series_type=close&time_period={time_period}&apikey={key}'

  # request data as json, convert to dict, pause request to avoid the data throttle
  r = requests.get(url)
  time.sleep(throttle)
  d = r.json()

  # extract to a df, add the indicator name, convert the index to datetime
  df = pd.DataFrame(d[f'Technical Analysis: {funct}']).T
  df.index = pd.to_datetime(df.index)

  # convert the data to float
  for col in df.columns:
    df[col] = df[col].astype('float')

  # check for missing data
  missing = df.isnull().any().sum()
  total = len(df)
  missing_pct = round(missing/total*100,2)


  # Print the results
  summary=['Technical Indicator',funct,str(total),str(missing),str(missing_pct)+'%','none']

  return {'summary':summary,'data':df}


## Combine Stock Data
+ price data
+ economic
+ technical

In [111]:
def get_consolidated_stock_data(symbol,key,config,outputsize='compact',throttle=30,dropna=True):
  """
  Pulls data from alpha advantage and consolidates
  API Limitations: 5 API requests per minute and 500 requests per day
  Inputs:
    symbol: stock ticker
    key   : api key
    config: dictionary which lists the economic, technical and commodities to pull
    outputsize: compact(latest 100) or full (up to 20 years of daily data)
    throttle: number of seconds to wait between api requests
    dropna: True/False, drops any records with nan
  Output:
    A dataframe with consolidated price data for the symbol + economic/technical
    indicators and commodity prices
  """

  # Result header and accumulator
  header = ['Type','Data','Total','Missing',' % ','Action']
  summary =[]

  # Get stock prices
  try:
    results  = get_ticker_data(symbol,key,outputsize,0)
    dff = results['data']
    summary.append(results['summary'])
    print(f'Complete:===>Ticker:{symbol}')
  except:
    print(f'Error:===>Ticker:{symbol}')



  # Get Commodity prices
  # ****************************************************************************
  for commodity in config['Commodities']:
    try:
      # get prices
      results = get_ticker_data(commodity,key,outputsize,throttle)
      df = results['data']
      summary.append(results['summary'])
      print(f'Complete:===>Commodity:{commodity}')


      # rename close to commodity name, remove unneeded columns and join with 
      # the stock prices by date
      df.rename(columns={'close':commodity},inplace=True)
      df.drop(['open','high','low','volume','symbol'],axis=1,inplace=True)
      dff = dff.join(df,how='left')
    except:
      print(f"Error===>Commodity:{commodity}")


  # Economic Indicators
  # ****************************************************************************
  # loop through the config to pull the requested data
  for indicator,values in config['Economic'].items():
    if indicator == 'TREASURY_YIELD':
      for tr in values:
        try:
          results = get_economic_indicators(indicator,key,interval=tr['interval'],maturity=tr['maturity'],throttle=throttle)
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}:{tr['maturity']}")

          df = results['data']
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": tr['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}:{tr['maturity']}")
   
    else: 
      # daily
      if values['interval']=='daily':
        try:
          results = get_economic_indicators(indicator,key,interval=values['interval'],throttle=throttle)
          df = results['data']
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}")

          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")
  
      else: 
        try:
          # monthly or weekly
          results = get_economic_indicators(indicator,key,throttle=throttle)
          summary.append(results['summary'])
          df = results['data']
          print(f"Complete:===>{indicator}")

          # reindex to daily, fill missing values forward
          days = pd.date_range(start = min(df.index),end =max(df.index),freq='D')
          df =df.reindex(days,method = 'ffill')
      
          # join with the other data
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")

  # # Technical Indicators
  # ****************************************************************************
  for indicator,values in config['Technical'].items():
    try:
      results = get_technical_indicators(symbol,indicator,key,values['interval'],values['time_period'],throttle)
      df = results['data']
      summary.append(results['summary'])

      dff = dff.join(df,how='left')
      print(f"Complete:===>{indicator}")
    except:
      print(f"Error===>{indicator}")

  
  # clean column names
  dff.rename(columns={"Real Upper Band":'b-upper',
                      "Real Lower Band":'b-lower',
                      "Real Middle Band":"b-middle",
                      "RSI":"rsi",
                      "MACD_Hist":"macd_hist",
                      "MACD_Signal":"macd_signal",
                      "MACD":"macd"
                      },inplace=True)
      

  # Fill in any missing data after joining all datasets
  dff.fillna(method='bfill',inplace=True,axis = 0)

  # drop rows with missing commodity prices
  if dropna:
    dff.dropna(how='any',inplace=True)

  # print the results table
  print("\n\n")
  print(tabulate(summary,header))

  return dff

## Pull Stock Data

In [106]:
# Get Consolidated Data
# https://etfdb.com/
# stock:  BLX.TO =  jan 2005, VMW = aug 2007
# S&P500: SPY = jan 1993
# gold:   GLD = nov 2004
# oil:    USO =apr 2006, OIL = apr 2011
# energy: PBD =  jun 2007 (clean), PBW=mar 2005 (cleaner oil & gas), XLE= dec 1998 (oil & gas)
# gas:    UNL = jan 2010, BOIL=oct 2011
# vol:    VXX = jan 2009
# TSX:    EWC = mar 1996

config ={'Economic':
         {'TREASURY_YIELD':[{'interval':'daily','maturity':'5year','name':'yield5y'},
                            {'interval':'daily','maturity':'10year','name':'yield10y'},
                            {'interval':'daily','maturity':'30year','name':'yield30y'},
                            {'interval':'daily','maturity':'3month','name':'yield3m'}
                            ],
          'FEDERAL_FUNDS_RATE':{'interval':'daily','name':'ir'},
          'NONFARM_PAYROLL':{'interval':'monthly','name':'nfp'},
          'UNEMPLOYMENT':{'interval':'monthly','name':'unemployment'},
          'CONSUMER_SENTIMENT':{'interval':'monthly','name':'cs'},
          'INFLATION_EXPECTATION':{'interval':'monthly','name':'infl'},
          },
         'Technical':{
           'BBANDS':{'interval':'daily','time_period':20},
           'RSI':{'interval':'daily','time_period':20},
           'MACD':{'interval':'daily','time_period':20}
           },
         'Commodities':['GLD','USO','SPY','PBD','XLE','EWC']
         }

# compact, full
stock = 'BLX.TO'
outputsize = 'full'
dff = get_consolidated_stock_data(stock,key,config,throttle=30,outputsize=outputsize,dropna=True)


# save to google drive
dff.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_{outputsize}.pickle')

# save to local drive
dff.to_pickle(f"market_data_{outputsize}.csv")
dff.to_csv(f'market_data_{outputsize}.pickle')
files.download(f"market_data_{outputsize}.csv")
files.download(f'market_data_{outputsize}.pickle')

Complete:===>Ticker:BLX.TO
Complete:===>Commodity:GLD
Complete:===>Commodity:USO
Complete:===>Commodity:SPY
Complete:===>Commodity:PBD
Complete:===>Commodity:XLE
Complete:===>Commodity:EWC
Complete===>TREASURY_YIELD:5year
Complete===>TREASURY_YIELD:10year
Complete===>TREASURY_YIELD:30year
Complete===>TREASURY_YIELD:3month
Complete===>FEDERAL_FUNDS_RATE
Complete===>NONFARM_PAYROLL
Complete===>UNEMPLOYMENT
Complete===>CONSUMER_SENTIMENT
Complete===>INFLATION_EXPECTATION
Complete===>BBANDS
Complete===>RSI
Complete===>MACD



Type                 Data                     Total    Missing   %      Action
-------------------  ---------------------  -------  ---------  ------  -----------
Ticker               BLX.TO                    4188          0  0.0%    none
Ticker               GLD                       4235          0  0.0%    none
Ticker               USO                       3886          0  0.0%    none
Ticker               SPY                       5504          0  0.0%    none
T

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Visualizations
+ Candlestick Plots + Economic Indicators
+ Technical Indicators + Interest Rates
+ Correlations between Features

In [123]:
def plot_candlestick(df,yearfrom=None):
    """
    Plots candleskick plots: open/close/high/low prices + volume
    Input:
      df      : A dataframe with the price data
      yearfrom: filters the df by year
    """

    # get ticker name
    name = df.symbol.unique()[0]

    # filter by year if provided
    if yearfrom is not None:
      df = df[df.index.year >= yearfrom]
    
    # Create subplots
    fig = make_subplots(rows=6, cols=1, 
                        shared_xaxes=True, 
                        subplot_titles=('Prices: Open/High/Low/Close','Volume','Non-Farm Payroll','Consumer Sentiment','Expected Inflation','Unemployment'),
                        vertical_spacing=0.07)
                   
    # OHLC Plot
    fig.add_trace(go.Candlestick(x=df.index, open=df.open, high=df.high,low=df.low, close=df.close, 
                name="Price",
                increasing_line_color= '#42BDA0', 
                decreasing_line_color= '#BD425F'), 
                row=1, col=1)
    
    
    # area chart for volume
    fig.add_trace(go.Scatter(x=df.index, y=df.volume, 
                fill='tozeroy',
                name='Volume',
                mode='none',
                fillcolor='rgba(156,189,66,.5)'),
                row=2,col=1)
    
    # Non-Farm Payrolls
    fig.add_trace(go.Scatter(x=df.index, y=df.nfp,name='Non-Farm Payroll',line_color='#BD425F'),
                  row=3,col=1)
    
    # Consumer Sentiment
    fig.add_trace(go.Scatter(x=df.index, y=df.cs,name='Consumer Sentiment',line_color='#42BDA0'),
                  row=4,col=1)

    # Expected Inflation    
    fig.add_trace(go.Scatter(x=df.index, y=df.infl,name='Expected Inflation',line_color='#6342BD'),
                  row=5,col=1)
    
    # Unemployment    
    fig.add_trace(go.Scatter(x=df.index, y=df.unemployment,name='Unemployment'),
                  row=6,col=1)

    # Do not show OHLC's rangeslider plot 
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.update_layout(template='plotly_white',
                      title = f"Price, Volume and Economic Indicators: {name}",
                      width = 1000,
                      height = 800)
          
    fig.show()

plot_candlestick(dff,2010)

In [126]:
dff.head()

,open,high,low,close,volume,symbol,GLD,OIL,SPY,PBD,VXX,yield5y,yield10y,yield30y,yield3m,ir,nfp,unemployment,cs,infl,b-upper,b-middle,b-lower,macd,macd_signal
2021-09-16,3613.55,3621.02,3576.87,3580.63,18405.3280,Ethereum,164.03,22.00,447.17,29.32,25.18,0.81,1.28,1.87,0.04,0.08,146856.0,5.2,81.2,4.7,4061.584546,3529.64,2997.695454,0.000000,0.000000
2021-09-15,3434.37,3616.16,3357.78,3613.55,428507.5278,Ethereum,167.83,22.00,447.88,29.56,25.43,0.81,1.28,1.87,0.04,0.08,146856.0,5.2,81.2,4.7,4061.584546,3529.64,2997.695454,2.626097,0.525219
2021-09-14,3285.09,3437.05,3269.95,3434.37,373634.9026,Ethereum,168.82,21.34,444.17,29.57,26.46,0.79,1.28,1.85,0.04,0.08,146856.0,5.2,81.2,4.7,4061.584546,3529.64,2997.695454,-9.639908,-1.507806
2021-09-13,3404.30,3430.30,3108.49,3285.35,647855.7199,Ethereum,167.74,21.41,446.58,29.55,26.09,0.81,1.33,1.91,0.06,0.08,146856.0,5.2,81.2,4.7,4061.584546,3529.64,2997.695454,-31.027805,-7.411806
2021-09-12,3267.10,3474.16,3231.10,3404.21,408858.9595,Ethereum,167.18,21.10,445.44,29.39,27.06,0.82,1.35,1.94,0.05,0.08,146856.0,5.2,81.2,4.7,4061.584546,3529.64,2997.695454,-37.949401,-13.519325


In [135]:
def plot_technicals(df,yearfrom=None):
  """
  Plots technical indicators
  Input:
    df: a dataframe with techical indicators
    yearfrom: filters the df by year
  """

  # get the symbol name
  name = df.symbol.unique()[0]

  # filter by year if provided
  if yearfrom is not None:
    df = df[df.index.year >= yearfrom]


  # make subplots
  fig = make_subplots(rows=2, cols=2,
                      shared_xaxes=True,
                      subplot_titles = ('Bollinger Bands','MACD','RSI','Interest Rates'),
                      vertical_spacing = 0.07)

  # boolinger bands
  # ********************************************************************************************
  fig.add_trace(go.Scatter(x=df.index,y=df['b-upper'],name='Bollinger-Upper',line_color="#DFEBF9"),row=1,col=1)
  fig.add_trace(go.Scatter(x=df.index,y=df['b-lower'],name='Bollinger-Lower',line_color='#DFEBF9',fill='tonexty'),row=1,col=1)
  fig.add_trace(go.Scatter(x=df.index,y=df['b-middle'],name='Closing Price',line_color='#297AD6'),row=1,col=1)


  # MACD
  # ********************************************************************************************
  fig.add_trace(go.Scatter(x=df.index,y=df['macd'],name='MACD',line_color='#F7DAC6'),row=1,col=2)
  fig.add_trace(go.Scatter(x=df.index,y=df['macd_signal'],name='MACD Signal',line_color='#E68A4C'),row=1,col=2)
  try:
    fig.add_trace(go.Bar(x=dff.index,y=df['macd_hist'],name='MACD Hist',marker_color='#E06D1F', marker_line_color='#E06D1F'),row=1,col=2)
  except:
    # skip mcad histogram for crypto
    print('')


  # # RSI
  # ********************************************************************************************
  fig.add_trace(go.Scatter(x=df.index,y=df['rsi'],name='RSI',line_color='#F00F3C'),row=2,col=1)

  fig.update_yaxes(range=[0, 100],row=2,col=1)


  # Interest Rates
  # add Fed Rate, then loop through the df columns to add any feature that is a yield
  # ********************************************************************************************
  fig.add_trace(go.Scatter(x=df.index, y=df.ir,name='Fed Funds Rate',line_color='#4B9D0C'),row=2,col=2)
  colors = ['#0E340F','#29541F','#5E9C53','#A3E090','#537455','#3B463E']
  rate_cols = [c for c in df.columns if 'yield' in c]
  for i,rate in enumerate(rate_cols):
    # Extract the name of the yield
    name = 'Yield:'+re.findall(r'\d+', rate)[0]+' Year'
    fig.add_trace(go.Scatter(x=df.index, y=df[rate],name=name,line_color = colors[i]),row=2,col=2)


  # Set template
  fig.update_layout(template = 'plotly_white',width= 1000,height=800,title ='Technical Indicators & Yields')
  fig.show()


plot_technicals(dff,2010)

In [136]:
def plot_distribution(df,feature):
  """
  Plot a historgram and bloxplot the input feature
  Input: 
    df:      a dataframe of features
    feature: a feature name
  Output:
    Plots the histogram and boxplot for the feature
  """

  # create subplots
  fig = make_subplots(rows=1, cols=2)

  # histogram
  fig.add_trace(
      go.Histogram(x=df[feature],name='histogram'),
      row=1, col=1
  )

  #box plot
  fig.add_trace(
      go.Box(y=df[feature],name='bloxplot'),
      row=1, col=2
  )

  #
  fig.update_layout(height=500, 
                    width=500, 
                    title_text=f"Distribution:{feature}",
                    template='plotly_white')
  fig.show()


plot_distribution(dff,'close')

In [137]:
def plot_corr(df):
  """
  Plots correlations in a heatmap
  Input: 
    df: a dataframe of features to plot
  """
  # subset columns
  # remove redundent columns (only keep one price, and the signal for technical indicators)
  cols = [c for c in df.columns if c not in ['open','high','low','b-upper','b-lower','macd_hist','macd','symbol']]
          
  # calculate correlations
  cm =df[cols].corr()

  # plot a heatmap of correlations
  fig = go.Figure()
  fig.add_trace(go.Heatmap(
        z = cm,
        x = cm.columns.values,
        y = cm.columns.values,
        colorscale = 'Bluyl'
      )
  )
  fig.update_layout(
      title_text="Feature Correlations", 
      title_x=0.5, 
      width=500, 
      height=500,
      yaxis_autorange='reversed',
      template = 'plotly_white'
  )

  fig.show()

plot_corr(dff)